In [ ]:
'''
inference.ipynb

Title: Soil Type Classification - Inference Notebook
Team: Team Cygnus
Authors: Vaibhav Sharma, Shreya Khantal, Prasanna Saxena
Best Model: 'soil_model.pth'

'''

In [ ]:
# inference.ipynb

# Step 1: Imports
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
import torch.nn.functional as F
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

# Step 2: Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Step 3: Define model architecture
class SoilClassifier(nn.Module):
    def __init__(self, model_name='resnet18', pretrained=False):
        super(SoilClassifier, self).__init__()
        self.backbone = models.resnet18(weights=None)
        num_ftrs = self.backbone.fc.in_features
        self.backbone.fc = nn.Linear(num_ftrs, 2)
        
    def forward(self, x):
        return self.backbone(x)

# Load model
model = SoilClassifier()
model.load_state_dict(torch.load("soil_model.pth", map_location=device))  # Make sure this file exists
model = model.to(device)
model.eval()

# Step 4: Define dataset
class SoilImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['file_path']
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, self.dataframe.iloc[idx]['image_id']

# Step 5: Define transforms (adjust mean/std if known)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4886, 0.3984, 0.3095], std=[0.1494, 0.1462, 0.1414])
])

# Step 6: Load test data
test_df = pd.read_csv("test_ids.csv")  # CSV with image_id column
test_df['file_path'] = test_df['image_id'].apply(lambda x: os.path.join("test", x))

test_dataset = SoilImageDataset(test_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Step 7: Inference
results = []
with torch.no_grad():
    for images, image_ids in test_loader:
        images = images.to(device)
        outputs = model(images)
        probs = F.softmax(outputs, dim=1)
        preds = torch.argmax(probs, dim=1)
        for image_id, pred, prob in zip(image_ids, preds.cpu(), probs[:,1].cpu()):
            results.append({
                'image_id': image_id,
                'label': int(pred),
                'soil_probability': float(prob)
            })

submission_df = pd.DataFrame(results)
submission_df.head()

# Step 8: Save submission
submission_df[['image_id', 'label']].to_csv("submission.csv", index=False)
print("Saved to submission.csv")
